# Kolhatkar et al parallels

In [182]:
# !pip3 install simpledorff
# !pip3 install krippendorff
# !pip3 install nltk
# !pip3 install py-readability-metrics
# !pip3 install spacy
# !pip3 install spacy-transformers
# !python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 51 kB 3.3 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 19.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 31.5 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 50.7 MB/s eta 0:00:01     |███████████████████████▍        | 655 kB 50.7 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 25.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 28.0 MB/s eta 0:00:01     |███████████████████▌            | 2.0 MB 28.0 MB/s eta 0:00:01
ERROR: huggingface-hub 0.1.2 has requirement packaging>=20.9, but you'll have packaging 20.4 which is incompatible.


In [301]:
# !pip3 show spacy
# !pip show spacy

Name: spacy
Version: 3.2.0
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /u/arunas/.local/lib/python3.8/site-packages
Requires: srsly, numpy, packaging, tqdm, thinc, pydantic, blis, requests, preshed, typer, cymem, spacy-loggers, langcodes, wasabi, setuptools, spacy-legacy, catalogue, jinja2, murmurhash, pathy
Required-by: spacy-transformers, en-core-web-sm
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.


In [302]:
import pandas as pd
import simpledorff
import krippendorff
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from readability import Readability
import spacy
nlp = spacy.load("en_core_web_sm")

nltk.download('punkt')

[nltk_data] Downloading package punkt to /u/arunas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [303]:
train = pd.read_csv('../data/processed/training_data.csv')
test = pd.read_csv('../data/processed/test_data.csv')
val = pd.read_csv('../data/processed/val_data.csv')

In [352]:
def modify_dataframe(dataset):
    dataset_ratings = dataset.filter(regex=('author_id|participantId|noteId|output|scaling|tweet_text|summary'))
    dataset_ratings = dataset_ratings.groupby(['noteId', 'participantId', 'summary', 'tweet_text', 'tweet_author_id']).size().to_frame(name = 'count').reset_index().merge(dataset_ratings)
    dataset_ratings['full_text'] = dataset_ratings.apply(lambda row: f'<BEG> {row["tweet_text"]} <SEP> {row["summary"]} <END>', axis=1)
    dataset_unique = pd.DataFrame(dataset_ratings['full_text'].unique())
    dataset_unique['full_text'] = dataset_unique[0]
    dataset_unique = dataset_unique.drop(0, 1)
    dataset_unique['ner_count'] = dataset_unique['full_text'].apply(lambda row: len(nlp(row).ents))
    dataset_ratings = pd.merge(dataset_ratings, dataset_unique, on='full_text')
    dataset_ratings['full_text_tokens'] = dataset_ratings.apply(lambda row: nltk.word_tokenize(row['full_text']), axis=1)
    dataset_ratings['full_text_average_word_length'] = dataset_ratings.apply(lambda row: np.mean([len(token) for token in row['full_text_tokens']]), axis=1)
    dataset_ratings["full_text_sentences"] = dataset_ratings.apply(lambda row: nltk.sent_tokenize(row['full_text']), axis=1)
    dataset_ratings['full_text_sentences_mean_word_count'] = dataset_ratings.apply(lambda row: np.mean([len(sentence.split(' ')) for sentence in row['full_text_sentences']]), axis=1)
    return dataset_ratings

# Analysis of crowdsourced ratings

In [304]:
train_ratings = train.filter(regex=('author_id|participantId|noteId|output|scaling|tweet_text|summary'))

In [305]:
# train.columns
train_ratings_columns = ['noteId', 'participantId', 'createdAtMillis', 'tweetId',
       'classification', 'believable', 'harmful', 'validationDifficulty',
       'misleadingOther', 'misleadingFactualError',
       'misleadingManipulatedMedia', 'misleadingOutdatedInformation',
       'misleadingMissingImportantContext', 'misleadingUnverifiedClaimAsFact',
       'misleadingSatire', 'notMisleadingOther',
       'notMisleadingFactuallyCorrect',
       'notMisleadingOutdatedButNotWhenWritten', 'notMisleadingClearlySatire',
       'notMisleadingPersonalOpinion', 'trustworthySources', 'summary',
       'tweet_text', 'tweet_author_id', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'output', 'scaling']


train_ratings = train_ratings.groupby(['noteId', 'participantId', 'summary', 'tweet_text', 'tweet_author_id']).size().to_frame(name = 'count').reset_index().merge(train_ratings)
train_ratings["full_text"] = train_ratings.apply(lambda row: f'<BEG> {row["tweet_text"]} <SEP> {row["summary"]} <END>', axis=1)

In [306]:
train_ratings["full_text"].head(1)

0    <BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank.    Here?s the city hall press release in the archives: http://officialsource...    <END>
Name: full_text, dtype: object

In [308]:
full_text_unique['full_text'] = pd.DataFrame(train_ratings['full_text'].unique())
full_text_unique['ner_count'] = full_text_unique['full_text'].apply(lambda row: len(nlp(row).ents))

In [313]:
train_ratings = pd.merge(train_ratings, full_text_unique, on='full_text')

In [315]:
train_ratings

,noteId,participantId,summary,tweet_text,tweet_author_id,count,output,scaling,full_text,0,ner_count
0,1352796878438424576,6B6171E803FBBF58397EC4A0AF76A2FF,"This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource...",Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB,1352756202984226816,7,0,0.142857,"<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>","<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>",3
1,1352796878438424576,6B6171E803FBBF58397EC4A0AF76A2FF,"This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource...",Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB,1352756202984226816,7,1,0.142857,"<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>","<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>",3
2,1352796878438424576,6B6171E803FBBF58397EC4A0AF76A2FF,"This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource...",Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB,1352756202984226816,7,1,0.142857,"<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>","<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>",3
3,1352796878438424576,6B6171E803FBBF58397EC4A0AF76A2FF,"This Tweet is partially true, but misleading. A look at the city archives shows that the image in th

In [316]:
train_ratings['full_text_tokens'] = train_ratings.apply(lambda row: nltk.word_tokenize(row['full_text']), axis=1)

In [317]:
# train_ratings["full_text_average_word_length"] = train_ratings.apply(lambda row: np.mean([len(token) * row['scaling'] for token in row['full_text_tokens']]), axis=1)

train_ratings["full_text_average_word_length"] = train_ratings.apply(lambda row: np.mean([len(token) for token in row['full_text_tokens']]), axis=1)

In [318]:
train_ratings["full_text_sentences"] = train_ratings.apply(lambda row: nltk.sent_tokenize(row['full_text']), axis=1)

In [319]:
# train_ratings['full_text_sentences_mean_word_count'] = train_ratings.apply(lambda row: np.mean([len(sentence.split(' ')) * row['scaling'] for sentence in row['full_text_sentences']]), axis=1)

train_ratings['full_text_sentences_mean_word_count'] = train_ratings.apply(lambda row: np.mean([len(sentence.split(' ')) for sentence in row['full_text_sentences']]), axis=1)

In [320]:
print(train_ratings.columns)
# print(train_ratings.apply(lambda row: print(row)))


Index([                             'noteId',
                             'participantId',
                                   'summary',
                                'tweet_text',
                           'tweet_author_id',
                                     'count',
                                    'output',
                                   'scaling',
                                 'full_text',
                                           0,
                                 'ner_count',
                          'full_text_tokens',
             'full_text_average_word_length',
                       'full_text_sentences',
       'full_text_sentences_mean_word_count'],
      dtype='object')


In [321]:
pd.set_option('display.max_colwidth', None)
train_ratings[['full_text', 'full_text_sentences', 'full_text_sentences_mean_word_count']].head(10)

,full_text,full_text_sentences,full_text_sentences_mean_word_count
0,"<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>","[<BEG> Coming from the mayor’s office soon., They want to convert all of our water fountains from still to sparkling!, https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading., A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank., Here?s the city hall press release in the archives: http://officialsource... <END>]",13.0
1,"<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>","[<BEG> Coming from the mayor’s office soon., They want to convert all of our water fountains from still to sparkling!, https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading., A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank., Here?s the city hall press release in the archives: http://officialsource... <END>]",13.0
2,"<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>","[<BEG> Coming from the mayor’s office soon., They want to convert all of our water fountains from still to sparkling!, https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading., A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank., Here?s the city hall press release in the archives: http://officialsource... <END>]",13.0
3,"<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>","[<BEG> Coming from the mayor’s office soon., They want to convert all of our water fountains from still to sparkling!, https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading., A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank., Here?s the city hall press release in the archives: http://officialsource... <END>]",13.0
4,"<BEG> Coming from the mayor’s office soon. They want to convert all of our water fountains from still to sparkling! https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading. A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank. Here?s the city hall press release in the archives: http://officialsource... <END>","[<BEG> Coming from the mayor’s office soon., They want to convert all of our water fountains from still to sparkling!, https://t.co/c4Fwb75lLB <SEP> This Tweet is partially true, but misleading., A look at the city archives shows that the image in the Tweet actually comes from a planned april fools day prank., Here?s the city hall press release in 

# Test data

In [353]:
train_ratings_from_function = modify_dataframe(train)

In [356]:
# train_ratings_from_function = train_ratings_from_function.filter(regex=('author_id|participantId|noteId|output|scaling|tweet_text|summary'))
train_ratings = train_ratings.drop(0, 1)
print(train_ratings.columns)
print(train_ratings_from_function.columns)
print(train_ratings.equals(train_ratings_from_function))

Index(['noteId', 'participantId', 'summary', 'tweet_text', 'tweet_author_id',
       'count', 'output', 'scaling', 'full_text', 'ner_count',
       'full_text_tokens', 'full_text_average_word_length',
       'full_text_sentences', 'full_text_sentences_mean_word_count'],
      dtype='object')
Index(['noteId', 'participantId', 'summary', 'tweet_text', 'tweet_author_id',
       'count', 'output', 'scaling', 'full_text', 'ner_count',
       'full_text_tokens', 'full_text_average_word_length',
       'full_text_sentences', 'full_text_sentences_mean_word_count'],
      dtype='object')
True


In [357]:
test_ratings_from_function = modify_dataframe(test)

In [ ]:
val_ratings_from_function = modify_dataframe(val)

# Length of tokens

In [ ]:
print(test_ratings.groupby('output', as_index=False)['full_text_sentences_mean_word_count'].mean())


# print(test_ratings['full_text_sentences_mean_word_count'].groupby(['output']).mean(), test_ratings['full_text_sentences_mean_word_count'].groupby(['output']).std())

# print(train_ratings['full_text_sentences_mean_word_count'].mean(), train_ratings['full_text_sentences_mean_word_count'].std())

# Argumentation features